In [26]:
placeholder_to_entity_dict = {
    'DIAGNOSIS': ["I10","I708","I25110"],
    'THRESHOLD': ["1.0","2.0","7.0"],
    'DRUG': "M01AE01",
    'AGE': ["16","42","66"],
    'SEVERITY': "Serious"
}

In [ ]:
from itertools import product
import re
from typing import Dict, List, Tuple


def replace_placeholders(
    question_template: str,
    query_template: str,
    placeholder_to_entity_dict: Dict[str, List[str] | str],
) -> List[Tuple[str, str]]:

    # Define the regex pattern for the chosen delimiter
    question_pattern = r'\{(.*?)\}'
    query_pattern = r'§(.*?)§'

    # Extract placeholders used in question and query
    placeholders_query = set(re.findall(query_pattern, query_template))
    placeholders_question = set(re.findall(question_pattern,question_template))


    placeholders = placeholders_query  # or placeholders_question, since they are equal
    # Build replacement lists for each placeholder
    replacements = []
    keys = []
    for key in placeholders:
        value = placeholder_to_entity_dict.get(key.upper())
        if value is None:
            continue  # Skip unknown placeholders
        if isinstance(value, list):
            replacements.append(value)
        else:
            replacements.append([value])
        keys.append(key)

    # Generate all combinations
    all_combinations = list(product(*replacements))

    results = []
    for combo in all_combinations:
        question = question_template
        query = query_template
        for key, val in zip(keys, combo):
                question = re.sub(f'{{{key}}}', val,question)
                query = re.sub(f'§{key}§', val, query)
        results.append((question, query))

    return results

In [23]:
import pandas as pd

expanded_templates_df = pd.read_csv('/home/mathiasyap/Code/university/phkg/MAI_Project_PHKG/tests/expanded_templates.csv')

In [24]:
expanded_templates_df.size

240

In [27]:
further_df = expanded_templates_df
# Expand each row into multiple rows for each combination of placeholders
expanded_rows = []
for idx, row in further_df.iterrows():
    outcomes = replace_placeholders(row['question_template'], row['sparql_template'], placeholder_to_entity_dict)
    for question, sparql in outcomes:
        expanded_rows.append({'question': question, 'sparql': sparql})
        expanded_rows[-1]['question_template'] = row['question_template']
        expanded_rows[-1]['sparql_template'] = row['sparql_template']
    
# Create a new DataFrame with the expanded results
results_df = pd.DataFrame(expanded_rows)
results_df

,question,sparql,question_template,sparql_template
0,What patients have received a diagnosis of I10?,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,What patients have received a diagnosis of {Di...,SELECT ?patient WHERE {\n ?patient sphn:hasDi...
1,What patients have received a diagnosis of I708?,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,What patients have received a diagnosis of {Di...,SELECT ?patient WHERE {\n ?patient sphn:hasDi...
2,What patients have received a diagnosis of I25...,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,What patients have received a diagnosis of {Di...,SELECT ?patient WHERE {\n ?patient sphn:hasDi...
3,Which patients have been identified with I10?,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,Which patients have been identified with {Diag...,SELECT ?patient WHERE {\n ?patient sphn:hasDi...
4,Which patients have been identified with I708?,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,Which patients have been identified with {Diag...,SELECT ?patient WHERE {\n ?patient sphn:hasDi...
...,...,...,...,...
265,Which patients have dealt with an adverse even...,SELECT ?patient WHERE {\n ?ae <http://www.w3....,Which patients have dealt with an adverse even...,SELECT ?patient WHERE {\n ?ae <http://www.w3....
266,Which patients have experienced an adverse eve...,SELECT ?patient WHERE {\n ?ae <http://www.w3....,Which patients have experienced an adverse eve...,SELECT ?patient WHERE {\n ?ae <http://www.w3....
267,Which patients have faced an adverse event tha...,SELECT ?patient WHERE {\n ?ae <http://www.w3....,Which patients have faced an adverse event tha...,SELECT ?patient WHERE {\n ?ae <http://www.w3....
268,Which patients have had an adverse event that ...,SELECT ?patient WHERE {\n ?ae <http://www.w3....,Which patients have had an adverse event that ...,SELECT ?patient WHERE {\n ?ae <http://www.w3....


In [14]:
results_df.size

1080

In [33]:
results_df['sparql'][0]

'SELECT ?patient WHERE {\n  ?patient sphn:hasDiagnosis ?diag .\n  ?diag sphn:hasCode ?code .\n  ?code sphn:hasValue I10 .\n}\n'

In [18]:
from pipeline.query_engine_component import QueryExecutorStep
query_executor = QueryExecutorStep(engine_name = "milleniumDB", graph_path="rdf_400_sphn_augmented_hybrid.nt"
, construct_graph=False)

['mdb:latest']
['docker-aseprite:latest']
['ghcr.io/avantlab/avantgraph:openaire-transport']
['ghcr.io/avantlab/avantgraph:openaire-energy']
['ghcr.io/avantlab/avantgraph:openaire-neuro']
['ghcr.io/avantlab/avantgraph:openaire-cancer']
['hello-world:latest']
['ghcr.io/avantlab/avantgraph:ckg']
['ghcr.io/avantlab/avantgraph:release-2024-01-31']


In [28]:
success_count = 0
fail_count = 0

def safe_query(q):
    global success_count, fail_count
    try:
        result = query_executor.query(q, path=False)
        success_count += 1
        return result
    except Exception:
        fail_count += 1
        return None

results_df['expected_result'] = results_df['sparql'].apply(safe_query)
print(f"Succeeded: {success_count}, Failed: {fail_count}")

Succeeded: 105, Failed: 165


In [29]:
unique_templates_with_no_result = results_df[results_df['expected_result'].isna()]['sparql_template'].unique()
unique_templates_with_no_result

array(['SELECT ?patient WHERE {\n  ?patient sphn:hasDiagnosis ?diag .\n  ?diag sphn:hasCode ?code .\n  ?code sphn:hasValue §Diagnosis§ .\n}\n',
       'SELECT ?patient WHERE {\n  ?event sphn:hasSubjectPseudoIdentifier ?patient .\n  ?event sphn:hasLabResult ?res .\n  ?res sphn:hasQuantityValue ?val .\n  FILTER(?val > §Threshold§)\n}\n',
       'SELECT ?patient WHERE {\n  ?presc sphn:hasSubjectPseudoIdentifier ?patient .\n  ?presc sphn:hasDrug ?drug .\n  ?drug sphn:hasCode ?code .\n  ?code sphn:hasValue §Drug§\n}\n',
       'SELECT (COUNT(DISTINCT ?patient) AS ?numPatients) WHERE {\n  ?patient <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDiagnosis> ?diag .\n  ?diag <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?code .\n  ?code <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodeValue> §Diagnosis§ .\n}\n',
       'SELECT ?patient WHERE {\n  ?presc <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?patient .\n  ?presc <https://www.biomedit.ch/rdf/sphn-

In [31]:

unique_templates_with_result = results_df[results_df['expected_result'].notna()]['sparql_template'].unique()
unique_templates_with_result

array(['SELECT ?patient WHERE {\n  ?patient <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasBirthDate> ?dob .\n  BIND(year(NOW()) - year(?dob) AS ?age)\n  FILTER(?age >= §Age§)\n}\n',
       'SELECT ?patient WHERE {\n  ?event <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?patient .\n  ?event <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabResult> ?res .\n  ?res <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasQuantityValue> ?val .\n  FILTER(?val < §Threshold§)\n}\n',
       'SELECT ?patient WHERE {\n  ?ae <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>\n       <https://www.biomedit.ch/rdf/sphn-schema/sphn/AdverseEvent> .\n  ?ae <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?patient .\n  ?ae <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSeverity> ?sev .\n  FILTER(?sev IN ("§Severity§"))\n}\n'],
      dtype=object)

In [20]:
results_df

,question,sparql,question_template,sparql_template,expected_result
0,What patients have received a diagnosis of I10?,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,What patients have received a diagnosis of {Di...,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,None
1,What patients have received a diagnosis of I708?,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,What patients have received a diagnosis of {Di...,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,None
2,What patients have received a diagnosis of I25...,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,What patients have received a diagnosis of {Di...,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,None
3,Which patients have been identified with I10?,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,Which patients have been identified with {Diag...,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,None
4,Which patients have been identified with I708?,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,Which patients have been identified with {Diag...,SELECT ?patient WHERE {\n ?patient sphn:hasDi...,None
...,...,...,...,...,...
265,Which patients have dealt with an adverse even...,SELECT ?patient WHERE {\n ?ae <http://www.w3....,Which patients have dealt with an adverse even...,SELECT ?patient WHERE {\n ?ae <http://www.w3....,[{'patient': http://kg-representation-ehr.org/...
266,Which patients have experienced an adverse eve...,SELECT ?patient WHERE {\n ?ae <http://www.w3....,Which patients have experienced an adverse eve...,SELECT ?patient WHERE {\n ?ae <http://www.w3....,[{'patient': http://kg-representation-ehr.org/...
267,Which patients have faced an adverse event tha...,SELECT ?patient WHERE {\n ?ae <http://www.w3....,Which patients have faced an adverse event tha...,SELECT ?patient WHERE {\n ?ae <http://www.w3....,[{'patient': http://kg-representation-ehr.org/...
268,Which patients have had an adverse event that ...,SELECT ?patient WHERE {\n ?ae <http://www.w3....,Which patients have had an adverse event that ...,SELECT ?patient WHERE {\n ?ae <http://www.w3....,[{'patient': http://kg-representation-ehr.org/...


In [ ]:
type(results)

NameError: name 'results' is not defined